In [10]:
import requests
import pandas as pd
import geojson
import math
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [2]:
url = "https://www.doaks.org/resources/coins/catalogue#b_start=0"
browser = webdriver.Chrome(executable_path = '/Users/webbc01/Desktop/ByzCoinsMap/chromedriver')
browser.get(url)
browser.find_element_by_id('__ac_name').send_keys('WebbC01')
browser.find_element_by_id('__ac_password').send_keys('CharlesW')
browser.find_element_by_css_selector('input[type=submit]').click()
try:
    myElem = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.TextileEntry')))
except TimeoutException:
    print("Loading took too much time!")
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
#get all options for the mints category
mints = soup.find('select', {'id': 'c11'})
#exclude all mints with 'disabled' attributes
f_mints = []
for mint in mints.children:
    if mint != '\n':
        if len(mint.attrs) < 4:
            f_mints.append(mint['value'])




In [3]:
print(f_mints)

['Abila', 'Alexandria', 'Alexandria ad Issum', 'Al-Jazira', 'Amida', 'Antioch', 'Arta', 'Baalbek/Heliopolis', 'Baisan/Skythopolis', 'Beneventum', 'Caesarea', 'Carthage', 'Catania', 'Cherson', 'Constantinople', 'Cyprus', 'Damascus', 'Fustat', 'Halab/Beroea', 'Hisn Kayfa', 'Ifriqya', 'Isaur(i)a', 'Italian Mint', 'Jerash/Gerasa', 'Jerusalem', 'Kyzikos', 'Lund', 'Magnesia', 'Merida', 'Military', 'Mytilene', 'Neapolis/Nablus', 'Nikomedeia', 'Philadelphia', 'Philippopolis', 'Ragusa', 'Ravenna', 'Rome', 'Sardinia', 'Sel=C3=A7uk/Theologos', 'Seleukeia', 'Sicily', 'Sis', 'Spain', 'Syracuse', 'Syrian Mint', 'Thessalonike', 'Ticinum', 'Trebizond', 'Turnovo', 'Uncertain Mint', 'Vidin', 'Yubna']


In [16]:
mint_coords = pd.read_excel('Mint-Locations.xlsx')
acc = []
for idx, row in mint_coords.iterrows():
    if math.isnan(row.Longitude) == False:
        temp_p = geojson.Point([row.Longitude, row.Latitude])
        temp_f = geojson.Feature(geometry=temp_p, properties={'Mint': row.Mint})
        acc.append(temp_f)
        
f_c = geojson.FeatureCollection(acc)

with open('mints.geojson', 'w') as f:
    geojson.dump(f_c, f)
    f.close()